**Revisiting Deep Hyperspectral Feature Extraction Networks via Gradient Centralized Convolution**

本实验是在源代码的基础上将输入维度从100改为了30

In [1]:
!pip install spectral

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.9/212.9 kB 5.5 MB/s eta 0:00:00


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, cohen_kappa_score
import spectral
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import copy
from scipy.io import loadmat
import math

In [15]:
class Conv3d_cd(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1,
                 padding=1, dilation=1, groups=1, bias=False, theta=0.7):

        super(Conv3d_cd, self).__init__() 
        self.conv = nn.Conv3d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding, dilation=dilation, groups=groups, bias=bias)
        self.theta = theta

    def forward(self, x):
        out_normal = self.conv(x)


        if math.fabs(self.theta - 0.0) < 1e-8:
            return out_normal 
        else:
            #pdb.set_trace()
            [C_out,C_in, kernel_size,kernel_size,depth] = self.conv.weight.shape
            kernel_diff = self.conv.weight.sum(2).sum(2)
            kernel_diff = kernel_diff[:, :, None, None]
            kernel_diff = kernel_diff.repeat(1,1,3,3,1)
            out_diff = F.conv3d(input=x, weight=kernel_diff, bias=self.conv.bias, stride=self.conv.stride, padding=self.conv.padding, groups=self.conv.groups)

            return out_normal - self.theta * out_diff

class CNN(nn.Module):
    def __init__(self, FM, Classes, patchsize, NC):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(
            Conv3d_cd(
                in_channels = 1,
                out_channels = FM,
                kernel_size = (3, 3, 7),
                stride = 1,
                padding = (0,0,0)
            ),
            nn.BatchNorm3d(FM),
            nn.ReLU(),
            nn.MaxPool3d(kernel_size=(1,1,2)),
#             nn.Dropout(0.5),
        )
        self.final_bands = (NC - 6) // 2
        
        self.conv2 = nn.Sequential(
            Conv3d_cd(FM, FM*2, (3, 3, 1 ), 1, (0,0,0)),
            nn.BatchNorm3d(FM*2),
            nn.ReLU(),
            nn.MaxPool3d(kernel_size=(1,1,2))

        )
        self.final_bands = (self.final_bands - 6) // 2
        self.conv3 = nn.Sequential(
            Conv3d_cd(FM*2, FM*4, (3, 3, 1), 1, (0,0,0)),
            nn.BatchNorm3d(FM*4),
            nn.ReLU(),
            nn.MaxPool3d(kernel_size=(1,1,2))

        )
        self.final_bands = (self.final_bands - 6) // 2
        
        self.final_patch_size = patchsize - 6
        
        self.out1 =  nn.Linear(4800, Classes)
        
    def forward(self, x1):
        #x1 = x1.unsqueeze(1)
        x1 = self.conv1(x1)
        #print(x1.shape)
        x1 = self.conv2(x1)
        #print(x1.shape)
        x1 = self.conv3(x1)
        #print(x1.shape)
        x1 = x1.reshape(x1.shape[0], -1)  # flatten the output of conv2 to (batch_size, 32 * 7 * 7)
        #print(x1.shape)
        out1 = self.out1(x1)
        return out1

In [16]:
# G2C-3DConv需要将维度设置大一些
# 第二个参数为classes
x = torch.randn(1, 1, 11, 11, 30)
net = CNN(16,16,15,100)
y = net(x)
print(y.shape)

torch.Size([1, 16])
